In [17]:
import tensorflow as tf
import pathlib
import cv2

# Load your pre-trained segmentation model
def weighted_crossentropy(class_weights):
      def loss_fn(y_true, y_pred):
          # Apply softmax activation to logits
          y_pred = tf.nn.softmax(y_pred, axis=-1)

          # Flatten both inputs
          y_true_f = tf.reshape(y_true, [-1])
          y_pred_f = tf.reshape(y_pred, [-1, tf.shape(y_pred)[-1]])

          # Calculate weighted cross-entropy loss
          loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=tf.cast(y_true_f, dtype=tf.int32), logits=y_pred_f)
          weighted_loss = tf.reduce_mean(tf.multiply(loss, tf.gather(class_weights, tf.cast(y_true_f, dtype=tf.int32))))

          return weighted_loss

      return loss_fn

segmentation_model = tf.keras.models.load_model('saved_seg_models/model2023-03-18 15_39_47.663348', compile=False)
segmentation_model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
      loss=weighted_crossentropy,
      metrics=["accuracy"],
    )

# Freeze the layers of the segmentation model
for layer in segmentation_model.layers:
    layer.trainable = False

# Load your training dataset and pass each image through the segmentation model to obtain its segmented output
train_dir = pathlib.Path('plantvillage')
batch_size = 32
img_height = 512
img_width = 512

training_dataset = tf.keras.utils.image_dataset_from_directory(
  train_dir,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)  # load your training dataset

segmented_training_dataset = []
for images, labels in training_dataset:
     segmentation_model.predict(images)
    # for index in range(len(images)):
    #     image = images[index]
    #     label = labels[index]
    # #     print(images.shape, label.shape)
    #     segmented_image = segmentation_model.predict(image[tf.newaxis, ...])
    #     segmented_training_dataset.append((segmented_image, label))

# Remove all pixels labeled as class 0 and 1 from the segmented output
segmented_training_dataset = tf.convert_to_tensor(segmented_training_dataset)
class_mask = tf.math.logical_and(tf.not_equal(segmented_training_dataset, 0), tf.not_equal(segmented_training_dataset, 1))
segmented_training_dataset = tf.where(class_mask, segmented_training_dataset, tf.zeros_like(segmented_training_dataset))

# Use the segmented output with class 0 and 1 removed as input to the segmentation model to extract features
feature_extraction_model = tf.keras.models.Sequential([
    segmentation_model.layers[0],  # input layer
    segmentation_model.layers[1],  # first convolutional layer
    segmentation_model.layers[2],  # second convolutional layer
    segmentation_model.layers[3]   # third convolutional layer
])

# Use the extracted features as input to the image classification model
classification_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(64, 64, 1)),  # flatten the extracted features
    tf.keras.layers.Dense(64, activation='relu'),     # fully connected layer
    tf.keras.layers.Dense(10, activation='softmax')   # output layer with 10 classes
])

# Train the classifier on the extracted features using your training dataset
classification_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
classification_model.fit(segmented_training_dataset, epochs=1)

# # Test the performance of the trained image classification model on a test dataset
# test_dataset = ...  # load your test dataset
# segmented_test_dataset = []
# for image in test_dataset:
#     segmented_image = segmentation_model.predict(image)
#     segmented_test_dataset.append(segmented_image)
# segmented_test_dataset = tf.convert_to


Found 18835 files belonging to 10 classes.


2023-03-21 16:25:49.231150: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


ResourceExhaustedError: Graph execution error:

Detected at node 'model/up_sampling2d_2/resize/ResizeBilinear' defined at (most recent call last):
    File "/opt/homebrew/Caskroom/miniconda/base/envs/project/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/homebrew/Caskroom/miniconda/base/envs/project/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/homebrew/Caskroom/miniconda/base/envs/project/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/project/lib/python3.10/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/opt/homebrew/Caskroom/miniconda/base/envs/project/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/opt/homebrew/Caskroom/miniconda/bas